In [20]:
# Import the necessary libraries
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import regexp_replace

import pandas as pd
import os

In [21]:
# Create a Spark session and set the configurations to access the AWS S3 bucket
spark = SparkSession.builder \
    .appName("Divvy Bikes - Data Cleaning Using Spark (From AWS S3 Bucket)") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,com.amazonaws:aws-java-sdk-bundle:1.11.563") \
    .config("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("fs.s3a.metrics.enabled", "false") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.DefaultAWSCredentialsProviderChain") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.memory.fraction", "0.8") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

#### Part 1. Get the data from AWS S3 Bucket and create a Spark DataFrame

In [22]:
# S3 bucket name
bucket_name = "divvydatacsv"

# Function to read CSV from S3
def read_csv_from_s3(file_name):
    s3_path = f"s3a://{bucket_name}/{file_name}"
    return spark.read.csv(s3_path, header=True, inferSchema=True)

# Path for local Parquet file
local_path = "local_data/divvy_tripdata_2022full.parquet"

# Check if local Parquet file exists
if not os.path.exists(local_path):
    # If not, read from S3 and save it as a local Parquet file
    months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
    dataframes = [read_csv_from_s3(f"divvy_tripdata_2022{month}.csv") for month in months]
    sdf = dataframes[0]
    for df in dataframes[1:]:
        sdf = sdf.union(df)
    
    # Save as Parquet file for faster reading in the future
    sdf.write.parquet(local_path)
    print("Data extracted from S3 is saved to a local Parquet file")
else:
    print("Local Parquet file already exists")

# Read the local Parquet file
sdf = spark.read.parquet(local_path)

Data extracted from S3 is saved to a local Parquet file


#### Part 2. Preview the data

In [23]:
# Print the schema
sdf.printSchema()

# Print the number of records
print(sdf.count())

# Show the first 5 records
sdf.show(5)

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- member_casual: string (nullable = true)

5667717
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|         start_lat|         start_

#### Part 3. General Data Cleaning

In [24]:
# Let's begin by checking trailing and leading whitespace
def has_whitespace(col):
    return (F.length(col) != F.length(F.trim(col)))

whitespace_check = [F.when(has_whitespace(F.col(c)), F.lit(True)).otherwise(F.lit(False)).alias(f"{c}_has_whitespace") 
                    for c in sdf.columns if sdf.schema[c].dataType == StringType()]

sdf_with_check = sdf.select("*", *whitespace_check)
rows_with_whitespace = sdf_with_check.filter(F.array_contains(F.array(*[f"{c}_has_whitespace" for c in sdf.columns if sdf.schema[c].dataType == StringType()]), True))

print("Rows with leading/trailing whitespace in the DataFrame:")
rows_with_whitespace.show(6, truncate=False)

# Count the rows with whitespace
original_count = rows_with_whitespace.count()

print(f"Number of rows with whitespace in the DataFrame: {original_count}")

Rows with leading/trailing whitespace in the DataFrame:
+----------------+-------------+-------------------+-------------------+---------------------------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+----------------------+----------------------------+---------------------------------+-------------------------------+-------------------------------+-----------------------------+----------------------------+
|ride_id         |rideable_type|started_at         |ended_at           |start_station_name                     |start_station_id|end_station_name|end_station_id|start_lat|start_lng|end_lat|end_lng|member_casual|ride_id_has_whitespace|rideable_type_has_whitespace|start_station_name_has_whitespace|start_station_id_has_whitespace|end_station_name_has_whitespace|end_station_id_has_whitespace|member_casual_has_whitespace|
+----------------+-------------+-------------------+-------------------+------------------------------

Number of rows with whitespace in the DataFrame: 328


In [25]:
# Now we know that there are multiple rows with whitespace, we need to use trim() to fix this
sdf = sdf.select([F.trim(F.col(c)).alias(c) if sdf.schema[c].dataType == StringType() else F.col(c) for c in sdf.columns])

In [26]:
# Since we've cleaned the data, and since we've updated 'sdf', let's check the whitespace again
sdf_with_check = sdf.select("*", *whitespace_check)
rows_with_whitespace = sdf_with_check.filter(F.array_contains(F.array(*[f"{c}_has_whitespace" for c in sdf.columns if sdf.schema[c].dataType == StringType()]), True))

print("Rows with leading/trailing whitespace in the DataFrame:")
rows_with_whitespace.show(truncate=False)

# Count the rows with whitespace in the original DataFrame
original_count = rows_with_whitespace.count()

print(f"Number of rows with whitespace in the DataFrame: {original_count}")

Rows with leading/trailing whitespace in the DataFrame:


+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+----------------------+----------------------------+---------------------------------+-------------------------------+-------------------------------+-----------------------------+----------------------------+
|ride_id|rideable_type|started_at|ended_at|start_station_name|start_station_id|end_station_name|end_station_id|start_lat|start_lng|end_lat|end_lng|member_casual|ride_id_has_whitespace|rideable_type_has_whitespace|start_station_name_has_whitespace|start_station_id_has_whitespace|end_station_name_has_whitespace|end_station_id_has_whitespace|member_casual_has_whitespace|
+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+----------------------+----------------------------+---------------------------------+-----------

Number of rows with whitespace in the DataFrame: 0


For the next data cleaning steps, let's utilize Spark SQL. You can utilize Spark SQL or use standard DataFrame operations (up to you), but I utilize both of them in this notebook for demonstration purposes.

In [27]:
# To utilize Spark SQL, we can start by creating a temporary view
sdf.createOrReplaceTempView("sdf_view")

In [28]:
# Count the number of null values in each column
null_counts_query = "SELECT " + ", ".join([f"SUM(CASE WHEN {c} IS NULL THEN 1 ELSE 0 END) AS {c}" for c in sdf.columns]) + " FROM sdf_view"
null_counts = spark.sql(null_counts_query)
null_counts.show()

+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+
|ride_id|rideable_type|started_at|ended_at|start_station_name|start_station_id|end_station_name|end_station_id|start_lat|start_lng|end_lat|end_lng|member_casual|
+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+
|      0|            0|         0|       0|            833064|          833064|          892742|        892742|        0|        0|   5858|   5858|            0|
+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+



In [29]:
# Now we know that there are null values, let's use Spark SQL to drop rows with any null values
sdf = spark.sql("SELECT * FROM sdf_view WHERE " + " AND ".join([f"{c} IS NOT NULL" for c in sdf.columns]))

# Show the number of records after dropping rows with null values
print(sdf.count())

4369360


In [30]:
# Update the temporary view since we have updated the Spark DataFrame
sdf.createOrReplaceTempView("sdf_view")

In [31]:
# According to Divvy Data website, trips below 60 seconds in length should be removed. 
# Therefore, let's create "ride_length" based on "started_at" and "ended_at" to calculate and investigate this
spark.sql("""
    SELECT *
    FROM (
        SELECT started_at, ended_at, (unix_timestamp(ended_at) - unix_timestamp(started_at)) AS ride_length 
        FROM sdf_view
    ) subquery
    WHERE ride_length < 60
    ORDER BY ride_length
""").show(6)

+-------------------+-------------------+-----------+
|         started_at|           ended_at|ride_length|
+-------------------+-------------------+-----------+
|2022-10-13 14:42:10|2022-10-13 11:53:28|     -10122|
|2022-06-07 19:14:47|2022-06-07 17:05:42|      -7745|
|2022-06-07 19:14:46|2022-06-07 17:07:45|      -7621|
|2022-11-06 01:58:11|2022-11-06 01:00:12|      -3479|
|2022-11-06 01:59:05|2022-11-06 01:02:03|      -3422|
|2022-11-06 01:57:21|2022-11-06 01:02:07|      -3314|
+-------------------+-------------------+-----------+
only showing top 6 rows



In [32]:
# Now we know that there are ride_length with less than 60 seconds (they even have minus numbers), it's time to clean them up
sdf = spark.sql("""
        SELECT *
        FROM (
            SELECT *, (unix_timestamp(ended_at) - unix_timestamp(started_at)) AS ride_length 
            FROM sdf_view
        ) subquery
        WHERE ride_length >= 60
        ORDER BY ride_length
    """)

sdf.show(6)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+-----------------+------------------+-------------+-----------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|        start_lat|         start_lng|          end_lat|           end_lng|member_casual|ride_length|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+------------------+-----------------+------------------+-------------+-----------+
|3CC6543FEE9EF143| classic_bike|2022-12-01 06:34:52|2022-12-01 06:35:52|Indiana Ave & Roo...|          SL-005|Indiana Ave & Roo...|        SL-005|        41.867888|        -87.623041|        41.867888|        -87.623041|       member|         60|
|167C3FC123F

In [33]:
# Update the temporary view since we have updated the Spark DataFrame
sdf.createOrReplaceTempView("sdf_view")

#### Part 4. Investigate Data Quality Issues (before more specific data cleaning)

In [34]:
# Let's check and investigate potential data quality issues. We can start by checking uppercase values from start_station_name 
# to find naming inconsistencies.
spark.sql("""
    SELECT *
    FROM sdf_view
    WHERE UPPER(start_station_name) = start_station_name
    ORDER BY start_station_name
""").show(6)

+----------------+-------------+-------------------+-------------------+------------------+--------------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+-----------+
|         ride_id|rideable_type|         started_at|           ended_at|start_station_name|    start_station_id|    end_station_name|end_station_id|         start_lat|         start_lng|           end_lat|           end_lng|member_casual|ride_length|
+----------------+-------------+-------------------+-------------------+------------------+--------------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+-----------+
|226015676A82060F|electric_bike|2022-06-22 15:08:17|2022-06-22 15:25:14|   WEST CHI-WATSON|DIVVY 001 - Wareh...|Milwaukee Ave & F...|           428|         41.924738|-87.70064133333334|             41.92|             -87.7|       casual|       10

I found something interesting above. Check the rows where start_station_name have the value "WEST CHI-WATSON". These same rows show the start_station_id as "DIVVY" something. Let's check further if all start_station_id that contain the text "DIVVY" are simple test stations or not.

In [35]:
# let's use pandas so we can display the full station names for the start_station_name
result_pd = spark.sql("""
    SELECT start_station_id, start_station_name
    FROM sdf_view
    WHERE start_station_id LIKE '%DIVVY%'
    ORDER BY start_station_name
""").toPandas()

pd.set_option('display.max_colwidth', None)

print(result_pd.head(6))

                     start_station_id start_station_name
0  DIVVY 001 - Warehouse test station    WEST CHI-WATSON
1  DIVVY 001 - Warehouse test station    WEST CHI-WATSON
2  DIVVY 001 - Warehouse test station    WEST CHI-WATSON
3                           DIVVY 001    WEST CHI-WATSON
4  DIVVY 001 - Warehouse test station            WestChi
5  DIVVY 001 - Warehouse test station            WestChi


From the latest finding above, one row has the value "DIVVY 001" for its start_station_id, while the other rows have the value "DIVVY 001 - Warehouse test station". While it's likely "DIVVY 001" is also a test station, we are not sure, so let's ignore it for now. Let's just make a note to clean up the rows later, where the start_station_id has the word "test", but not specifically "DIVVY".

In [36]:
# Next, let's check the uppercase values from end_station_name
spark.sql("""
    SELECT *
    FROM sdf_view
    WHERE UPPER(end_station_name) = end_station_name
    ORDER BY start_station_name
""").show(6)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------------+---------+----------+------------------+------------------+-------------+-----------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|      end_station_id|start_lat| start_lng|           end_lat|           end_lng|member_casual|ride_length|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------------+---------+----------+------------------+------------------+-------------+-----------+
|E5B1B30C08A03D7B| classic_bike|2022-06-04 15:28:21|2022-06-04 15:46:03|   Adler Planetarium|           13431|DIVVY CASSETTE RE...|DIVVY CASSETTE RE...|41.866095|-87.607267|41.895543469489944|-87.60292053222655|       casual|       1062|
|C6E1B2965BCED6B6| classic_bike|2022-08-02 23:36

Another interesting finding here. The above result shows that for the rows that contain the text "DIVVY" in their end_station_name, they also start with the text "DIVVY" for their end_station_id. Let's investigate this further.

In [37]:
# Just like before, let's use pandas so we can display the full station names (but this time for the end_station_name)
result_pd = spark.sql("""
    SELECT end_station_id, end_station_name
    FROM sdf_view
    WHERE end_station_id LIKE '%DIVVY%' OR end_station_name LIKE '%DIVVY%'
    ORDER BY end_station_name
""").toPandas()

pd.set_option('display.max_colwidth', None)

print(result_pd.head(10))

                         end_station_id                      end_station_name
0  DIVVY CASSETTE REPAIR MOBILE STATION  DIVVY CASSETTE REPAIR MOBILE STATION
1  DIVVY CASSETTE REPAIR MOBILE STATION  DIVVY CASSETTE REPAIR MOBILE STATION
2  DIVVY CASSETTE REPAIR MOBILE STATION  DIVVY CASSETTE REPAIR MOBILE STATION
3  DIVVY CASSETTE REPAIR MOBILE STATION  DIVVY CASSETTE REPAIR MOBILE STATION
4  DIVVY CASSETTE REPAIR MOBILE STATION  DIVVY CASSETTE REPAIR MOBILE STATION
5  DIVVY CASSETTE REPAIR MOBILE STATION  DIVVY CASSETTE REPAIR MOBILE STATION


There are rows with value "DIVVY CASSETTE REPAIR MOBILE STATION", which means they are just used for maintenance, not for actual trips. This needs to be filtered as well later.

In [38]:
# Now, let's check the lowercase values from start_station_name
spark.sql("""
    SELECT *
    FROM sdf_view
    WHERE LOWER(start_station_name) = start_station_name
    ORDER BY start_station_name
""").show(6)

+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+-----------+
|ride_id|rideable_type|started_at|ended_at|start_station_name|start_station_id|end_station_name|end_station_id|start_lat|start_lng|end_lat|end_lng|member_casual|ride_length|
+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+-----------+
+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+-----------+



There's nothing here, let's continue.

In [39]:
# Let's check the lowercase values from end_station_name
spark.sql("""
    SELECT *
    FROM sdf_view
    WHERE LOWER(end_station_name) = end_station_name
    ORDER BY end_station_name
""").show(6)

+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+-----------+
|ride_id|rideable_type|started_at|ended_at|start_station_name|start_station_id|end_station_name|end_station_id|start_lat|start_lng|end_lat|end_lng|member_casual|ride_length|
+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+-----------+
+-------+-------------+----------+--------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+-----------+



Nothing to see here as well. Let's continue. 

Now that we know some rows have the word "test" in start_station_id (from one of the previous investigations), we should check all the station columns for the word "test". Let's check regardless of their case sensitivity.

In [40]:
# Again, let's use pandas so we can display the full station names
result_pd = spark.sql("""
    SELECT start_station_id, start_station_name, end_station_id, end_station_name
    FROM sdf_view
    WHERE LOWER(start_station_name) LIKE '%test%'
        OR LOWER(end_station_name) LIKE '%test%'
        OR LOWER(start_station_id) LIKE '%test%'
        OR LOWER(end_station_id) LIKE '%test%'
    ORDER BY end_station_name DESC
""").toPandas()

pd.set_option('display.max_colwidth', None)

print(result_pd.head(10))

                      start_station_id     start_station_name end_station_id  \
0  Hubbard Bike-checking (LBS-WH-TEST)  Base - 2132 W Hubbard   TA1307000138   
1  Hubbard Bike-checking (LBS-WH-TEST)  Base - 2132 W Hubbard          13285   
2   DIVVY 001 - Warehouse test station        WEST CHI-WATSON          13285   
3  Hubbard Bike-checking (LBS-WH-TEST)  Base - 2132 W Hubbard          13221   
4  Hubbard Bike-checking (LBS-WH-TEST)  Base - 2132 W Hubbard          13221   
5  Hubbard Bike-checking (LBS-WH-TEST)  Base - 2132 W Hubbard          13221   
6  Hubbard Bike-checking (LBS-WH-TEST)  Base - 2132 W Hubbard          13221   
7  Hubbard Bike-checking (LBS-WH-TEST)  Base - 2132 W Hubbard          13432   
8  Hubbard Bike-checking (LBS-WH-TEST)  Base - 2132 W Hubbard          13432   
9  Hubbard Bike-checking (LBS-WH-TEST)  Base - 2132 W Hubbard            637   

             end_station_name  
0       Wood St & Webster Ave  
1  Wood St & Taylor St (Temp)  
2  Wood St & Taylor St 

Apart from the issues with the word "test", there are also station names that end with "(Temp)", which probably means "Temporary". We will have to fix the naming inconsistencies here later. Let's investigate this further. Let's check for the word "temp" in the station columns.

In [41]:
# Let's use pandas so we can check the full station names
result_pd = spark.sql("""
    SELECT start_station_id, start_station_name, end_station_id, end_station_name
    FROM sdf_view
    WHERE LOWER(start_station_name) LIKE '%temp%'
        OR LOWER(end_station_name) LIKE '%temp%'
        OR LOWER(start_station_id) LIKE '%temp%'
        OR LOWER(end_station_id) LIKE '%temp%'
    ORDER BY start_station_name
""").toPandas()

pd.set_option('display.max_colwidth', None)

print(result_pd.head(6))

  start_station_id   start_station_name end_station_id  \
0     KA1504000155  2112 W Peterson Ave          13259   
1            15491        63rd St Beach   TA1308000026   
2            13028    900 W Harrison St          13285   
3            13028    900 W Harrison St          13285   
4            13028    900 W Harrison St          13285   
5            13028    900 W Harrison St          13285   

                     end_station_name  
0  California Ave & Francis Pl (Temp)  
1      Wentworth Ave & 24th St (Temp)  
2          Wood St & Taylor St (Temp)  
3          Wood St & Taylor St (Temp)  
4          Wood St & Taylor St (Temp)  
5          Wood St & Taylor St (Temp)  


I found multiple station names with "(Temp)", we will have to fix the station names later. Another potential data quality issue is the presence of the asterisk character in the station columns. Let's check for the presence of the asterisk character in the station columns.

In [42]:
# Again, let's use pandas to check the full station names. This time, to check the names that contain asterisks (*)
result_pd = spark.sql("""
    SELECT start_station_id, start_station_name, end_station_id, end_station_name
    FROM sdf_view
    WHERE LOWER(start_station_name) LIKE '%*%'
        OR LOWER(end_station_name) LIKE '%*%'
        OR LOWER(start_station_id) LIKE '%*%'
        OR LOWER(end_station_id) LIKE '%*%'
""").toPandas()

pd.set_option('display.max_colwidth', None)

print(result_pd.head(6))

  start_station_id       start_station_name end_station_id  \
0     chargingstx3  Green St & Randolph St*   TA1307000120   
1     chargingstx3  Green St & Randolph St*   chargingstx3   
2     chargingstx4     Morgan St & Lake St*        20246.0   
3     chargingstx3  Green St & Randolph St*          13053   
4    chargingstx07  Green St & Madison Ave*          13053   
5     chargingstx3  Green St & Randolph St*          13409   

                end_station_name  
0          Green St & Madison St  
1        Green St & Randolph St*  
2         N Green St & W Lake St  
3     Green St & Washington Blvd  
4     Green St & Washington Blvd  
5  Sangamon St & Washington Blvd  


Many rows have asterisks in the station names, we need to fix them later.

#### Part 5 - Specific Data Cleaning (to fix the issues from part 4)

In [43]:
# Clean and fix all the data quality issues we have identified from part 4
sdf = spark.sql("""
    SELECT * 
    FROM sdf_view
    WHERE upper(start_station_name) != start_station_name
      AND upper(end_station_name) != end_station_name
      AND lower(start_station_name) NOT LIKE '%test%'
      AND lower(end_station_name) NOT LIKE '%test%'
      AND lower(start_station_id) NOT LIKE '%test%'
      AND lower(end_station_id) NOT LIKE '%test%'
      AND lower(start_station_name) NOT LIKE 'divvy cassette repair mobile station'
      AND lower(end_station_name) NOT LIKE 'divvy cassette repair mobile station'
      AND lower(start_station_id) NOT LIKE 'divvy cassette repair mobile station'
      AND lower(end_station_id) NOT LIKE 'divvy cassette repair mobile station'
""")

sdf = sdf.withColumn("start_station_name", regexp_replace("start_station_name", "\\*", "")) \
                 .withColumn("start_station_name", regexp_replace("start_station_name", "\\(Temp\\)", "")) \
                 .withColumn("end_station_name", regexp_replace("end_station_name", "\\*", "")) \
                 .withColumn("end_station_name", regexp_replace("end_station_name", "\\(Temp\\)", ""))


In [44]:
# Update the temporary view since we have updated the Spark DataFrame
sdf.createOrReplaceTempView("sdf_view")

# Also, show the cleaned data
sdf.show(5)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+-----------------+-------------+-----------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|        start_lat|        start_lng|          end_lat|          end_lng|member_casual|ride_length|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------------+-----------------+-----------------+-----------------+-------------+-----------+
|86221D7038654BC9| classic_bike|2022-12-08 20:30:09|2022-12-08 20:31:09|Avondale Ave & Ir...|           15624|Avondale Ave & Ir...|         15624|        41.953393|       -87.732002|        41.953393|       -87.732002|       member|         60|
|C88FF4AA14FA1B8B|el

In [45]:
# Test 1 - Making sure that the data has been properly cleaned, we should check the new 'sdf_view'. 
# We know station id 'chargingstx3' previously  had station name of 'Green St & Randolph St*', 
# so now we should see 'Green St & Randolph St' instead. 
result_pd = spark.sql("""
    SELECT start_station_id, start_station_name
    FROM sdf_view
    WHERE start_station_id = 'chargingstx3'
""").toPandas()

pd.set_option('display.max_colwidth', None)

print(result_pd.head(3))

  start_station_id      start_station_name
0     chargingstx3  Green St & Randolph St
1     chargingstx3  Green St & Randolph St
2     chargingstx3  Green St & Randolph St


In [46]:
# Test 2 - We know station id '13285' previously had station name of 'Wood St & Taylor St (Temp)', 
# so now we should see 'Wood St & Taylor St' instead.
result_pd = spark.sql("""
    SELECT end_station_id, end_station_name
    FROM sdf_view
    WHERE end_station_id = '13285'
""").toPandas()

pd.set_option('display.max_colwidth', None)

print(result_pd.head(3))

  end_station_id      end_station_name
0          13285  Wood St & Taylor St 
1          13285  Wood St & Taylor St 
2          13285  Wood St & Taylor St 


In [47]:
# Test 3 - We know station id 'DIVVY CASSETTE REPAIR MOBILE STATION' previously existed. 
# The outcome of this test should return an empty dataframe.
result_pd = spark.sql("""
    SELECT end_station_id, end_station_name
    FROM sdf_view
    WHERE end_station_id = 'DIVVY CASSETTE REPAIR MOBILE STATION'
""").toPandas()

pd.set_option('display.max_colwidth', None)

print(result_pd.head(3))

Empty DataFrame
Columns: [end_station_id, end_station_name]
Index: []


In [48]:
# Test 4 - We had some rows with station id that contain the text 'DIVVY 001 - Warehouse test station' 
# They shouldn't exist anymore after we cleaned the data. The outcome of this test should return an empty dataframe.
result_pd = spark.sql("""
    SELECT start_station_id, start_station_name
    FROM sdf_view
    WHERE start_station_id = 'DIVVY 001 - Warehouse test station'
""").toPandas()

pd.set_option('display.max_colwidth', None)

print(result_pd.head(3))

Empty DataFrame
Columns: [start_station_id, start_station_name]
Index: []


In [49]:
# All good! Time to stop Spark
spark.stop()